In [ ]:
# default_exp algorithm

# Inverse Design
> Method by Google X

In [ ]:
# exporti
from typing import NamedTuple

import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from fastcore.basics import patch_to

In [ ]:
UNASSIGNED = 0
VOID = 1
SOLID = 2
PIXEL_IMPOSSIBLE = 3
PIXEL_EXISTING = 4
PIXEL_POSSIBLE = 5
PIXEL_REQUIRED = 6
TOUCH_REQUIRED = 7
TOUCH_INVALID = 8
TOUCH_EXISTING = 9
TOUCH_VALID = 10
TOUCH_FREE = 11
TOUCH_RESOLVING = 12

In [ ]:
# export
class Design(NamedTuple):
    design: jnp.ndarray
    void_pixels: jnp.ndarray
    solid_pixels: jnp.ndarray
    void_touches: jnp.ndarray
    solid_touches: jnp.ndarray

In [ ]:
# export
def new_design(shape):
    return Design(
        design=jnp.zeros(shape, dtype=jnp.int8).at[:,:].set(VOID),
        void_pixels=jnp.zeros(shape, dtype=jnp.int8).at[:,:].set(PIXEL_POSSIBLE),
        solid_pixels=jnp.zeros(shape, dtype=jnp.int8).at[:,:].set(PIXEL_POSSIBLE),
        void_touches=jnp.zeros(shape, dtype=jnp.int8).at[:,:].set(TOUCH_VALID),
        solid_touches=jnp.zeros(shape, dtype=jnp.int8).at[:,:].set(TOUCH_VALID),
    )

In [ ]:
def circular_brush(diameter):
    radius = diameter / 2
    X, Y = jnp.mgrid[-radius:radius:1j*diameter,-radius:radius:1j*diameter]
    _int = lambda x: jnp.array(x, dtype=int)
    brush = _int(X)**2 + _int(Y)**2 < radius**2
    return brush

plt.imshow(circular_brush(16));

In [ ]:
# export
_cmap = ListedColormap( colors={ UNASSIGNED: "#929292", VOID: "#cbcbcb", SOLID: "#515151", PIXEL_IMPOSSIBLE: "#8dd3c7", PIXEL_EXISTING: "#ffffb3", PIXEL_POSSIBLE: "#bebada", PIXEL_REQUIRED: "#fb7f72", TOUCH_REQUIRED: "#00ff00", TOUCH_INVALID: "#7fb1d3", TOUCH_EXISTING: "#fdb462", TOUCH_VALID: "#b3de69", TOUCH_FREE: "#fccde5", TOUCH_RESOLVING: "#e0e0e0", }.values(), name="cmap" )
def visualize(design):
    nx, ny = design.design.shape
    fig, axs = plt.subplots(1, 5, figsize=(15,3*nx/ny))
    for i, title in enumerate(design._fields):
        ax = axs[i]
        ax.set_title(title.replace("_", " "))
        ax.pcolormesh(design[i], cmap=_cmap, vmin=UNASSIGNED, vmax=TOUCH_RESOLVING)
        ax.set_yticks(jnp.arange(nx), ["" for i in range(nx)])
        ax.set_xticks(jnp.arange(ny), ["" for i in range(ny)])
        ax.set_yticks(jnp.arange(nx)+0.5, [f"{i}" for i in range(nx)], minor=True)
        ax.set_xticks(jnp.arange(ny)+0.5, [f"{i}" for i in range(ny)], minor=True)
        ax.set_xlim(0, ny)
        ax.set_ylim(nx, 0)
        ax.grid(visible=True, which="major", c="k")

@patch_to(Design)
def _repr_html_(self):
    visualize(self)
    return ""
    
design = new_design((6, 8)) 
design

In [ ]:
def add_void_touch(design, brush, pos):
    return Design(
        design=design.design,
        void_pixels=design.void_pixels,
        solid_pixels=design.solid_pixels,
        void_touches=design.void_touches.at[pos].set(TOUCH_EXISTING),
        solid_touches=design.solid_touches,
    )

In [ ]:
brush = circular_brush(13)
add_void_touch(design, brush, (0, 6))